# Solar Azimuth and Zenith Tracker

## Chapter 1

This is the first app of a series, where i create simple apps to improve my programming skills.

An application which tracks the sun according to the latitude and longtitude or the address (defined by google) of the location. Based on stackoverflow answers.

In [35]:
#import astropy
import astropy.coordinates as coord
import astropy.units as u
from astropy.time import Time
#from astropy.coordinates import get_sun, AltAz, EarthLocation

#import utc time now
from datetime import datetime,timezone
now_utc = datetime.now(timezone.utc)

#ask user if coordinates or address
while True:
    try:
        selector = int(input('Enter: 1 for address, 2 for coordinates, 0 for defaults: '))
        break
    except:
        print('Not Valid, Enter: 1 for address, 2 for coordinates, 3 for defaults: ')
print(selector)

Enter: 1 for address, 2 for coordinates, 0 for defaults: 1
1


In [36]:
#Option 0: defaults
if selector == 0: 
    longt,latit,hei = 0,0,0
    print('Longtitude:',longt , 'Latitude:',latit,'Height:',hei)
    
#Option 1: ask user an address
#Work in progress for wrong addresses
if selector == 1:
    name = input('Enter an address: ')
    if name == '':
        name = 'Stirling UK'
    print('The address is:', name)

#Option 2: ask user an address
if selector == 2 :
    try:
        longt = int(input('Enter longtitude or 0 for default: '))
        if longt == 0 : longt = 0
        latit = int(input('Enter latitude or 0 for default: '))
        if latit == 0 : latit = 0
        hei = int(input('Enter height or 0 for default: '))
        if hei == 0 : hei = 0
        print('Longtitude:',longt , 'Latitude:',latit,'Height:',hei)
    except:
        longt,latit,hei = 0,0,0
        print('Longtitude:',longt , 'Latitude:',latit,'Height:',hei)

Enter an address: 
The address is: Stirling UK


In [42]:
sun_time = Time(now_utc) #UTC time
if selector == 1:
    print('Selector: ',selector)
    loc = coord.EarthLocation.of_address('name') # anything the google geocoding API resolves
if (selector == 2) or (selector == 0):
    print('Selector: ',selector)
    loc = coord.EarthLocation(lon=longt,lat=latit,height=hei)

print('Geocentric coordinates <<Earth diameter = 6.371km>>',loc)

altaz = coord.AltAz(obstime=sun_time, location=loc)
zen_ang = coord.get_sun(sun_time).transform_to(altaz).zen
zen_alt = coord.get_sun(sun_time).transform_to(altaz).alt

print('Sun Altitude Degrees (above 0 day)',zen_alt)
print('Sun Azimuth Degrees              :',zen_ang)

1
Geocentric coordinates <<Earth diameter = 6.371km>> (-1081932.44719753, 6285690.01465716, -12400.55449409) m
Sun Altitude Degrees (above 0 day) -65d40m18.0338s
Sun Altitude Degrees (above 0 day) -65d40m18.0188s
Sun Azimuth Degrees              : 155d40m18.0188s
